In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
import csv
from scipy import optimize
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split


from numpy import linalg as LA
import scipy.sparse as sparse
from scipy.sparse import csc_matrix
from scipy.sparse import dia_matrix
import itertools
from itertools import combinations
import operator

import math
import scipy.integrate as integrate

In [2]:
# def MM(t, pop,IC):
#     #IC is the 
#     a_r, a_s, a_m, K, e, f = 1.1, 2.0, 1.1, 10**6, 0.00001, 0.00001
#     R, S, M = pop
#     return [a_r*R*(1-(R+S+M)/K),
#             a_s*S*(1-(R+S+M)/K) - e*S*R - f*S*M,
#             a_m*M*(1-(R+S+M)/K) + e*S*R + f*S*M]

# tspan = np.linspace(0, 20, num=2001)
# dt = 0.01
# ini = [10, 10000, 0]
# sol = integrate.solve_ivp(RSM, [tspan[0], tspan[-1]], ini, method='RK45', t_eval=tspan)

# plt.plot(sol.t, sol.y[0].T, label='R')
# plt.plot(sol.t, sol.y[1].T, label='S')
# plt.plot(sol.t, sol.y[2].T, label='M')
# plt.legend()
# plt.show()
# # print(sol.y.shape)

In [3]:
Q=[0,0,1]
# m,n=Q.shape
P=np.zeros(len(Q))
# P
for q in range(len(Q)):
    if Q[q]==0:
        P[q]=1
P

array([1., 1., 0.])

In [4]:
co=[2]
co*np.ones(len(Q))

array([2., 2., 2.])

In [5]:
def IfElse(Q,point,counterPoint):
    #if Q, POINT, COUNTERPOINT ARE arrays???; len(point) should equal or greater than len(Q)!
    y=point;
    P=np.zeros(len(Q))
    # P is 'not Q or ~Q'
    for q in range(len(Q)):
        if Q[q]==0:
            P[q]=1
    if len(counterPoint)==1:
        counterPoint = counterPoint*np.ones(len(Q))

    for p in range(len(P)):
            
        if P[p]==1:
            y[p]=counterPoint[p]
    
    return y
            

In [6]:
#test
Q=np.array([0,0,5,0])
point=np.array([2,4,3,1,7])
counterPoint=np.array([9])
# counterPoint
IfElse(Q, point, counterPoint)

array([9, 9, 3, 9, 7])

In [17]:
np.array([9])

array([9])

In [7]:
def Fun_MarPas(x,beta,gamma):
    topSpec = (1 + np.sqrt(beta))**2
    botSpec = (1 - np.sqrt(beta))**2
    Q=(topSpec-x)*(x-botSpec)
    for q in range(len(Q)):
        if Q[q]>0:
            Q[q]=1
        else:
            Q[q]=0
    point = np.sqrt((topSpec-x)*(x-botSpec))/(beta*x)/(2*np.pi)  
    InitialMar = IfElse(Q,point,np.array([0])) #'0' number used to replace others
    if np.all(gamma != 0):
        return np.power(list(x),list(gamma))*InitialMar
    else:
        return InitialMar
    
    
    
#### separate
def MarPas(x,beta):
    topSpec = (1 + np.sqrt(beta))**2
    botSpec = (1 - np.sqrt(beta))**2
    Q=(topSpec-x)*(x-botSpec)
    for q in range(len(Q)):
        if Q[q]>0:
            Q[q]=1
        else:
            Q[q]=0
    point = np.sqrt((topSpec-x)*(x-botSpec))/(beta*x)/(2*np.pi)  
    InitialMar = IfElse(Q,point,np.array([0])) #########9#####0
#     if np.all(gamma != 0):
#         return np.power(list(x),list(gamma))*InitialMar
#     else:
#         return InitialMar
    return InitialMar


In [49]:
#test
# beta=np.array([0.1, 0.2, 0.3])
# # gamma=np.array([0.1, 0.1, 0.2])
# gamma=np.array([0, 0.1, 0.2])
# x=np.array([0.3, 0.4, 0.3])
# point=np.array([2,4])
# Fun_MarPas(x,beta,gamma)
# # IfElse(Q,point,0)

###test separate
beta=np.array([0.1, 0.2, 0.3])
gamma=np.array([0.1, 0.1, 0.2])
# gamma=np.array([0, 0.1, 0.2])
x=np.array([0.3, 0.4, 0.3])
point=np.array([2,4])
MarPas(x,beta)
# IfElse(Q,point,0)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in sqrt


array([9.        , 0.79577472, 0.79084727])

In [32]:
def incMarPas(x0,beta,gamma):
    #beta is an array including one (****or more****not true) number; 0<beta<=1 
    #x0 is an array with multiple numbers!!!
#     for b in beta: 
#         if b > 1:
#             raise Exception('dimension error')
    topSpec = (1 + np.sqrt(beta))**2
#     botSpec = (1 - np.sqrt(beta))**2
    x2=lambda x: Fun_MarPas(x,beta,gamma)
    I1 = np.zeros([len(x0)])#I1 is integrate collection
    I2 = np.zeros([len(x0)])
    for i in range(len(x0)):
        X0=x0[i]
        RES = integrate.quad(x2,np.array([X0]), topSpec)
        I1[i] = RES[0]
        I2[i] = RES[1]
#     return integrate.quad(x2, x0, topSpec)  #the first part of return is integrate
    return I1

In [10]:
# Fun_MarPas(x,beta,gamma)
# incMarPas(x,beta,gamma)
x=np.array([0.1])
beta=np.array([0.22])
gamma=np.array([0])
I=integrate.quad(lambda xx: Fun_MarPas(xx,beta,gamma), x, (1 + np.sqrt(beta))**2)
I[0]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in sqrt
  # Remove the CWD from sys.path while we load stuff.


2.6372516236082255

In [33]:
#test
#all inputs are one value/number rather than multiple numbers in a array
incMarPas(x,beta,gamma)

array([1.        , 0.62953116, 0.31631693, 0.10435709, 0.        ])

In [12]:
#if inputs are arrarys including multiple numbers?
def incMarPas2(x0,beta,gamma):
    #beta is an array including one or more numbers; 0<beta<=1
#     for b in beta: 
#         if b > 1:
#             raise Exception('dimension error')
    topSpec = (1 + np.sqrt(beta))**2   #is an array including multiple numbers if beta is an array including multiple numbers
        

    I1 = np.zeros([len(beta)])
    I2 = np.zeros([len(beta)])
    for i in range(len(beta)):
        BETA = beta[i]
        GAMMA = gamma[i]
        x2=lambda x: Fun_MarPas(x,np.array([BETA]),np.array([GAMMA]))
        X0=x0[i]
        TOPSPEC=topSpec[i]
        RES = integrate.quad(x2,np.array([X0]), np.array([TOPSPEC]))
        I1[i] = RES[0]
        I2[i] = RES[1]
    return I1, I2

In [13]:
b=np.array([0.22,0.1,0.2])
(1 + np.sqrt(b))**2

array([2.15808315, 1.73245553, 2.09442719])

In [14]:
incMarPas2(x,beta,gamma)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in sqrt
  # Remove the CWD from sys.path while we load stuff.


(array([2.63725162]), array([3.18701003e-08]))

In [16]:
# incMarPas2(b,b,b)

In [43]:
def MedianMarcenkoPastur(beta):
    #beta should be one number array!!!here
#     MarPas = lambda x: 1-incMarPas(x,beta,np.array([0]))
    lobnd = (1-np.sqrt(beta))**2
    hibnd = (1+np.sqrt(beta))**2
    change = 1
    while change and (hibnd - lobnd > 0.001):
        change = 0;
        x = np.linspace(start = lobnd, stop = hibnd, num = 5)
        y = np.zeros(len(x))
        for i in range(len(x)):
            y[i] = 1-incMarPas(x[i],beta,0)
        if any(y<0.5):
            lobnd = max(x[y < 0.5])
            change = 1
        if any(y>0.5):
            hibnd = min(x[y > 0.5])
            change = 1
    return (hibnd+lobnd)/2
    

In [44]:
# test
MedianMarcenkoPastur(beta)

array([0.92616194])

In [53]:
def optimal_SVHT_coef_sigma_known(beta):
    #lambda_star
    #omit ensuring beta
    w = (8*beta)/(beta+1+np.sqrt(beta**2+14*beta+1))
    lambda_star = np.sqrt(2*(beta+1)+w)
    return lambda_star

In [54]:
# test
optimal_SVHT_coef_sigma_known(beta)

array([1.5816484 , 1.7048151 , 1.80739223])

In [55]:
def optimal_SVHT_coef_sigma_unknown(beta):
    coef = optimal_SVHT_coef_sigma_known(beta)
    MPmedian = np.zeros(len(beta))
    for i in range(len(beta)):
        MPmedian[i] = MedianMarcenkoPastur(beta)
    return coef/np.sqrt(MPmedian)

In [57]:
# test
beta=np.array([0.22])#MedianMarcenkoPastur can only be applied to one number beta
optimal_SVHT_coef_sigma_unknown(beta)

array([1.79413181])